In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import mapping
import jenkspy
import numpy as np

import warnings
warnings.simplefilter(action='ignore')

In [2]:
#read superfund site data in
tri_in = gpd.read_file('../data/top-tri-releases.geojson')
tri = tri_in.copy()
tri

,TRIFD,FACILITY_NAME,STREET_ADDRESS,CITY,COUNTY,ST,ZIP,BIA,TRIBE,LATITUDE,...,TOTAL2010,TOTAL2011,TOTAL2012,TOTAL2013,TOTAL2014,TOTAL2015,TOTAL2016,TOTAL2017,TOTAL2018,geometry
0,00603HWLTTSTATE,HEWLETT-PACKARD CARIBE BV SITE,HWY 110 N KM 5.1,AGUADILLA,AGUADILLA MUNICIPIO,PR,603,,,18.434600,...,4.749170e+03,6.432610e+03,5.638600e+03,6.801900e+03,1.046200e+03,1.000000e-01,2.000000e-01,0.000000e+00,0.000000e+00,POINT (-67.15519 18.43460)
1,00608DCRBNRD3KM,IDI CARIBE INC,PR3 KM 151.8 AGUIRRE,SALINAS,SALINAS MUNICIPIO,PR,751,,,17.972778,...,1.258338e+04,1.323538e+04,1.426338e+04,1.619038e+04,1.528838e+04,1.554038e+04,1.476838e+04,1.372638e+04,1.314338e+04,POINT (-66.23194 17.97278)
2,00610BXTRHROAD4,EDWARDS LIFESCIENCES TECHNOLOGY SARL,STATE RD 402 KM 1.4 N LAS MARIAS IND US TRIAL ...,ANASCO,ANASCO MUNICIPIO,PR,610,,,18.293900,...,1.000000e+03,1.000000e+03,1.473000e+03,1.411000e+03,1.580160e+03,1.702870e+03,2.297090e+03,5.990000e+02,5.800000e+02,POINT (-67.13660 18.29390)
3,00610LLRGNGPOBO,ALLERGAN MEDICAL OPTICS,RD 402 LAS MARIAS GPO BOX 1408,ANASCO,ANASCO MUNICIPIO,PR,6104108,,,18.286290,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,POINT (-67.13591 18.28629)
4,00612PRPCMPR681,PREPA-CAMBALACHE COMBUSTION TURBINE PLANT,PR-681 KM 0.5,ARECIBO,ARECIBO MUNICIPIO,PR,612,,,18.471100,...,7.850000e+01,6.350000e+01,2.490000e+01,3.160000e+01,5.590000e+01,8.210000e+01,5.180000e+01,9.560000e+01,4.220000e+01,POINT (-66.69940 18.47110)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13509,99752RDDGP90MIL,RED DOG OPERATIONS,90 MILES N OF KOTZEBUE,KOTZEBUE,NORTHWEST ARCTIC BOROUGH,AK,99752,,,68.061507,...,7.773957e+08,9.893925e+08,8.140033e+08,9.063858e+08,1.124385e+09,5.612221e+08,7.769739e+08,1.082414e+09,8.856829e+08,POINT (-162.85549 68.06151)
13510,99801CRLSK331CL,COEUR ALASKA INC KENSINGTON GOLD PROJECT,3031 CLINTON DR,JUNEAU,JUNEAU BOROUGH,AK,99801,,,58.867490,...,9.420530e+04,1.273746e+04,1.080831e+06,2.010958e+06,1.985383e+06,2.169999e+06,1.951777e+06,2.226975e+06,2.212952e+06,POINT (-135.10476 58.86749)
13511,99801KNNCT13401,HECLA GREENS CREEK MINING CO,13401 GLACIER HWY,JUNEAU,JUNEAU BOROUGH,AK,99801,,,58.081802,...,4.718547e+07,4.704384e+07,4.875493e+07,5.061587e+07,4.155703e+07,2.995106e+07,4.178036e+07,5.869635e+07,5.558960e+07,POINT (-134.64121 58.08180)
13512,99835LSKPLMILE6,ALASKA PULP CORP,4600 SAWMILL CREEK RD,SITKA,SITKA,AK,998359798,,,57.047640,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,POINT (-135.23290 57.04764)


In [3]:
values = tri[['TOTAL1987',
         'TOTAL1988',
         'TOTAL1989',
         'TOTAL1990',
         'TOTAL1991',
         'TOTAL1992',
         'TOTAL1993',
         'TOTAL1994',
         'TOTAL1995',
         'TOTAL1996',
         'TOTAL1997',
         'TOTAL1998',
         'TOTAL1999',
         'TOTAL2000',
         'TOTAL2001',
         'TOTAL2002',
         'TOTAL2003',
         'TOTAL2004',
         'TOTAL2005',
         'TOTAL2006',
         'TOTAL2007',
         'TOTAL2008',
         'TOTAL2009',
         'TOTAL2010',
         'TOTAL2011',
         'TOTAL2012',
         'TOTAL2013',
         'TOTAL2014',
         'TOTAL2015',
         'TOTAL2016',
         'TOTAL2017',
         'TOTAL2018']].to_numpy()

values

array([[0.00000000e+00, 0.00000000e+00, 2.21000000e+04, ...,
        2.00000000e-01, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.47683800e+04, 1.37263800e+04, 1.31433800e+04],
       [2.72140000e+05, 2.84986000e+05, 2.57625000e+05, ...,
        2.29709000e+03, 5.99000000e+02, 5.80000000e+02],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.17803610e+07, 5.86963499e+07, 5.55895970e+07],
       [1.38750000e+06, 1.52406000e+06, 1.58130000e+06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.49827000e+06, 4.83824000e+06, 5.15345000e+06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [4]:
values = values.flatten()
values

array([    0.,     0., 22100., ...,     0.,     0.,     0.])

In [5]:
roots = np.sqrt(values)  
roots

array([  0.        ,   0.        , 148.66068747, ...,   0.        ,
         0.        ,   0.        ])

In [6]:
divisor = 300000
radius = np.divide(roots, divisor)
radius

array([0.        , 0.        , 0.00049554, ..., 0.        , 0.        ,
       0.        ])

In [7]:
multiple = 3779.5280352161
radiusPx = np.multiply(radius, multiple)
radiusPx

array([0.        , 0.        , 1.87289079, ..., 0.        , 0.        ,
       0.        ])

In [17]:
print(np.sort(radiusPx))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
radiusPx

array([0.        , 0.        , 1.87289079, ..., 0.        , 0.        ,
       0.        ])

In [18]:
np.amax(radiusPx)

422.4485391545018

In [ ]:
breaks = jenkspy.jenks_breaks(radiusPx, nb_class=5)
breaks